In [1]:
import tensorflow as tf
import keras as k
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras import backend as k

In [3]:
# Cargamos el modelo
iv3 = InceptionV3()

In [4]:
# Información del modelo, aparecen todas las capas del modelo. Es un modelo bestial con 23.851.784 parámetros
print(iv3.summary())

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [5]:
'''
Cargamos imagenes y la preprocesamos para que esté de la forma deseada por el modelo:
(299px x 299px) y las intensidades entre [0-1]
'''
from keras.preprocessing import image
x = image.img_to_array(image.load_img('./Imagenes/Stratocaster.jpg', target_size = (299, 299)))
# Ponemos la intensidad de los píxeles entre [-1 +1]
x /= 255 # Deja las intensidades entre [0-1]
x -= 0.5 # Deja las intensidades entre [-0.5 +0.5], sirve para centrar los valores
x *= 2 # Se dejan las intensidades entre [-1 +1]
print(x.shape)
'''
La entrada al modelo requiere de un tensor que tiene las siguientes dimensiones (n, 299, 299, 3)
La dimensión n sirve para indicar al modelo cuantas imágenes le vamos a meter
'''
n = 1
x = x.reshape([n, x.shape[0], x.shape[1], x.shape[2]])
print(x.shape)

(299, 299, 3)
(1, 299, 299, 3)


In [6]:
'''
Vamos a hacer una predicción. La predicción es un tensor (1, 1000), que indica la distribución de probabilidad
para cada una de las 1000 objetos que detecta el modelo
'''
y = iv3.predict(x)
'''
Para poder saber a qué objeto corresponde cada una de las posiciones de y usamos decode_predictions. Como se ve, 
a detectado que la imagen es una 'electric_guitar'.
'''
decode_predictions(y)

[[('n03272010', 'electric_guitar', 0.9832207),
  ('n02676566', 'acoustic_guitar', 0.0032207002),
  ('n03929660', 'pick', 0.0011115792),
  ('n04296562', 'stage', 0.00044155837),
  ('n02787622', 'banjo', 0.0002170482)]]

### Ataques adversarios

Los ataques adversarios tratan de intentar hacer pequeñas modificaciones en las imágenes, de forma que se engañe al modelo. Las modificaciones que se hacen a las imágenes no son visibles a simple vista. Esto se hace mediante un proceso de aprendizaje, en el cual lo que se pretende ir modificando los píxeles de la imagen de entrada para engañar al modelo. El modelo de reconocimiento de imágenes iv3 no se modifica, ya que se quiere que este funcione correctamente, lo que se entrena es el sistema de modificación de imágenes.

In [7]:
'''
Nos basamos en el modelo iv3 para hacer el sistema de modificación de imágenes. Para empezar cogemos la capa
de entrada y de salida del modelo. -1 indica la última posición de un array
'''
inp_layer = iv3.layers[0].input
out_layer = iv3.layers[-1].output
'''
Vamos a intentar confundir al modelo de forma que envez de detectar una guitarra eléctrica detecte un limón 
limon: clase 951
'''
target_class = 951
'''
Le decimos que la última capa, la que dice a la clase que pertenece la imagen, que maximice la probabilidad de la clase
que sea un limón. 
'''
loss = out_layer[0, target_class] # 0 indica la clase que ha predecido con más probabilidad
'''
Calculamos el gradiente entre el error (loss) y la var de entrada
Indica en que proporción hay que modificar los pesos del modificador de imágenes
'''
grad = k.gradients(loss, inp_layer)[0] 

'''
Función keras que va a optimizar el entrenamiento del modificador de imágenes. Tiene como entrada inp_layer y 
y k.learning_phase() es para indicar que estamos enfase estamos (0->entrenamiento). 
Como salida los gradientes y loss
'''
optimize_gradients = k.function([inp_layer, k.learning_phase()], [grad, loss]) 

# Vamos a comparar el utilizar o no pert al modificar la imagen
adv_pert = np.copy(x) # Para no modificar la imagen original
adv = np.copy(x)

In [8]:
cost = 0.0
# Entrenamiento con pert: mientras que el coste sea menor que 0.95 sigue entrenando
while cost < 0.95:
    '''
    El 0 indica que estamos en entrenamiento, es la entrada a la función k.learning_phase()
    gr son las direcciones de hacia dónde tenemos que modificar los píxeles de la imágen para 
    optimizar el sistema de modificación de imágenes
    '''
    gr, cost = optimize_gradients([adv, 0]) 
    # Sumamos a nuestra imagen los valores de los gradientes de forma que la modificamos en las direcciones deseadas
    adv += gr
    '''
    Para que la moficiación en la imagen sea imperceptible. Clip hace que cuando un valor pasa el umbral
    lo deja en el valor máximo o mínimo definido
    '''
    print('Target cost: ', cost)
hacked_img = np.copy(adv)


Target cost:  5.012696e-06
Target cost:  5.0135204e-06
Target cost:  5.014897e-06
Target cost:  5.0176022e-06
Target cost:  5.020598e-06
Target cost:  5.0237004e-06
Target cost:  5.026821e-06
Target cost:  5.029935e-06
Target cost:  5.033146e-06
Target cost:  5.036369e-06
Target cost:  5.0395834e-06
Target cost:  5.0427307e-06
Target cost:  5.045984e-06
Target cost:  5.0492013e-06
Target cost:  5.052431e-06
Target cost:  5.0556687e-06
Target cost:  5.0589347e-06
Target cost:  5.0622048e-06
Target cost:  5.0654357e-06
Target cost:  5.068737e-06
Target cost:  5.072007e-06
Target cost:  5.0753306e-06
Target cost:  5.078572e-06
Target cost:  5.0818207e-06
Target cost:  5.085059e-06
Target cost:  5.0883377e-06
Target cost:  5.0916237e-06
Target cost:  5.094893e-06
Target cost:  5.0981926e-06
Target cost:  5.10144e-06
Target cost:  5.1047214e-06
Target cost:  5.107946e-06
Target cost:  5.1112256e-06
Target cost:  5.114528e-06
Target cost:  5.1177794e-06
Target cost:  5.121024e-06
Target cost

Target cost:  6.045474e-06
Target cost:  6.0493853e-06
Target cost:  6.0532616e-06
Target cost:  6.057127e-06
Target cost:  6.0610523e-06
Target cost:  6.064996e-06
Target cost:  6.0689554e-06
Target cost:  6.0728876e-06
Target cost:  6.0768284e-06
Target cost:  6.0807483e-06
Target cost:  6.084694e-06
Target cost:  6.0886546e-06
Target cost:  6.092565e-06
Target cost:  6.0965417e-06
Target cost:  6.1005094e-06
Target cost:  6.1044716e-06
Target cost:  6.108432e-06
Target cost:  6.112451e-06
Target cost:  6.1164633e-06
Target cost:  6.120452e-06
Target cost:  6.1244987e-06
Target cost:  6.1284736e-06
Target cost:  6.1324986e-06
Target cost:  6.1364703e-06
Target cost:  6.1404985e-06
Target cost:  6.144573e-06
Target cost:  6.1485725e-06
Target cost:  6.152612e-06
Target cost:  6.1567043e-06
Target cost:  6.1607866e-06
Target cost:  6.1648834e-06
Target cost:  6.169017e-06
Target cost:  6.173125e-06
Target cost:  6.1772075e-06
Target cost:  6.181284e-06
Target cost:  6.1853757e-06
Targe

Target cost:  7.3715046e-06
Target cost:  7.376444e-06
Target cost:  7.381401e-06
Target cost:  7.38623e-06
Target cost:  7.3911533e-06
Target cost:  7.3960373e-06
Target cost:  7.400901e-06
Target cost:  7.4057843e-06
Target cost:  7.410645e-06
Target cost:  7.415504e-06
Target cost:  7.4204145e-06
Target cost:  7.425241e-06
Target cost:  7.4301283e-06
Target cost:  7.4350087e-06
Target cost:  7.439927e-06
Target cost:  7.444816e-06
Target cost:  7.449667e-06
Target cost:  7.4546297e-06
Target cost:  7.459542e-06
Target cost:  7.464401e-06
Target cost:  7.469215e-06
Target cost:  7.474037e-06
Target cost:  7.4789227e-06
Target cost:  7.48368e-06
Target cost:  7.4884833e-06
Target cost:  7.4932477e-06
Target cost:  7.4980953e-06
Target cost:  7.502858e-06
Target cost:  7.5076605e-06
Target cost:  7.5124212e-06
Target cost:  7.517218e-06
Target cost:  7.5220114e-06
Target cost:  7.526818e-06
Target cost:  7.5316125e-06
Target cost:  7.536392e-06
Target cost:  7.5411326e-06
Target cost: 

Target cost:  8.9784935e-06
Target cost:  8.984353e-06
Target cost:  8.990198e-06
Target cost:  8.996044e-06
Target cost:  9.0019075e-06
Target cost:  9.00779e-06
Target cost:  9.013595e-06
Target cost:  9.019441e-06
Target cost:  9.025336e-06
Target cost:  9.031176e-06
Target cost:  9.037021e-06
Target cost:  9.042877e-06
Target cost:  9.048763e-06
Target cost:  9.054585e-06
Target cost:  9.060376e-06
Target cost:  9.06627e-06
Target cost:  9.072172e-06
Target cost:  9.078069e-06
Target cost:  9.083945e-06
Target cost:  9.089765e-06
Target cost:  9.095643e-06
Target cost:  9.101508e-06
Target cost:  9.1073925e-06
Target cost:  9.113274e-06
Target cost:  9.119119e-06
Target cost:  9.125023e-06
Target cost:  9.130877e-06
Target cost:  9.136728e-06
Target cost:  9.142612e-06
Target cost:  9.148408e-06
Target cost:  9.154338e-06
Target cost:  9.160202e-06
Target cost:  9.166085e-06
Target cost:  9.172025e-06
Target cost:  9.177928e-06
Target cost:  9.183847e-06
Target cost:  9.1898e-06
Ta

Target cost:  1.1058373e-05
Target cost:  1.1066784e-05
Target cost:  1.1075163e-05
Target cost:  1.1083448e-05
Target cost:  1.10917435e-05
Target cost:  1.110007e-05
Target cost:  1.1108431e-05
Target cost:  1.1116775e-05
Target cost:  1.11251875e-05
Target cost:  1.113347e-05
Target cost:  1.114177e-05
Target cost:  1.1149988e-05
Target cost:  1.1158184e-05
Target cost:  1.11663685e-05
Target cost:  1.1174472e-05
Target cost:  1.1182697e-05
Target cost:  1.1190998e-05
Target cost:  1.1199309e-05
Target cost:  1.120768e-05
Target cost:  1.1216008e-05
Target cost:  1.1224407e-05
Target cost:  1.1232783e-05
Target cost:  1.1241224e-05
Target cost:  1.1249692e-05
Target cost:  1.1258136e-05
Target cost:  1.1266709e-05
Target cost:  1.12752305e-05
Target cost:  1.1283711e-05
Target cost:  1.1292249e-05
Target cost:  1.1300672e-05
Target cost:  1.1309103e-05
Target cost:  1.131754e-05
Target cost:  1.1325901e-05
Target cost:  1.1334312e-05
Target cost:  1.1342717e-05
Target cost:  1.13511

...

Target cost:  3.5919893e-05
Target cost:  3.5968056e-05
Target cost:  3.601679e-05
Target cost:  3.606559e-05
Target cost:  3.611419e-05
Target cost:  3.6162946e-05
Target cost:  3.621147e-05
Target cost:  3.6260135e-05
Target cost:  3.6309048e-05
Target cost:  3.6358237e-05
Target cost:  3.640712e-05
Target cost:  3.6456324e-05
Target cost:  3.6505295e-05
Target cost:  3.6554724e-05
Target cost:  3.660408e-05
Target cost:  3.665381e-05
Target cost:  3.670325e-05
Target cost:  3.675298e-05
Target cost:  3.680297e-05
Target cost:  3.6852798e-05
Target cost:  3.690249e-05
Target cost:  3.6952373e-05
Target cost:  3.700229e-05
Target cost:  3.705211e-05
Target cost:  3.7101832e-05
Target cost:  3.71516e-05
Target cost:  3.720171e-05
Target cost:  3.7252026e-05
Target cost:  3.730244e-05
Target cost:  3.73532e-05
Target cost:  3.7404257e-05
Target cost:  3.745504e-05
Target cost:  3.750585e-05
Target cost:  3.755653e-05
Target cost:  3.7607442e-05
Target cost:  3.7658356e-05
Target cost:  

Target cost:  5.6062425e-05
Target cost:  5.6159315e-05
Target cost:  5.625488e-05
Target cost:  5.634991e-05
Target cost:  5.6443736e-05
Target cost:  5.6538665e-05
Target cost:  5.6632944e-05
Target cost:  5.672729e-05
Target cost:  5.6820696e-05
Target cost:  5.6914185e-05
Target cost:  5.700771e-05
Target cost:  5.710171e-05
Target cost:  5.7196386e-05
Target cost:  5.7291287e-05
Target cost:  5.7386475e-05
Target cost:  5.7481364e-05
Target cost:  5.7576442e-05
Target cost:  5.767144e-05
Target cost:  5.776634e-05
Target cost:  5.7860147e-05
Target cost:  5.795482e-05
Target cost:  5.8050027e-05
Target cost:  5.8145004e-05
Target cost:  5.823958e-05
Target cost:  5.8334823e-05
Target cost:  5.8431968e-05
Target cost:  5.852985e-05
Target cost:  5.862694e-05
Target cost:  5.8725913e-05
Target cost:  5.8825328e-05
Target cost:  5.8924714e-05
Target cost:  5.902409e-05
Target cost:  5.912364e-05
Target cost:  5.9222428e-05
Target cost:  5.9321083e-05
Target cost:  5.9418962e-05
Targe

Target cost:  9.867762e-05
Target cost:  9.888129e-05
Target cost:  9.908659e-05
Target cost:  9.9294375e-05
Target cost:  9.950452e-05
Target cost:  9.971555e-05
Target cost:  9.992829e-05
Target cost:  0.00010014074
Target cost:  0.00010035239
Target cost:  0.00010056787
Target cost:  0.00010078264
Target cost:  0.00010099542
Target cost:  0.000101209764
Target cost:  0.00010142712
Target cost:  0.00010164328
Target cost:  0.00010186179
Target cost:  0.00010207979
Target cost:  0.00010230183
Target cost:  0.00010252639
Target cost:  0.000102751794
Target cost:  0.000102977705
Target cost:  0.000103204184
Target cost:  0.000103429404
Target cost:  0.000103649974
Target cost:  0.000103870356
Target cost:  0.00010408946
Target cost:  0.00010430851
Target cost:  0.000104528226
Target cost:  0.00010474821
Target cost:  0.0001049706
Target cost:  0.00010519511
Target cost:  0.00010541792
Target cost:  0.00010564379
Target cost:  0.00010587085
Target cost:  0.00010609785
Target cost:  0.000

Target cost:  0.00020489891
Target cost:  0.0002056013
Target cost:  0.00020630533
Target cost:  0.00020699976
Target cost:  0.00020770547
Target cost:  0.00020839686
Target cost:  0.00020910059
Target cost:  0.00020980777
Target cost:  0.00021052634
Target cost:  0.00021125318
Target cost:  0.0002119739
Target cost:  0.00021269704
Target cost:  0.00021340969
Target cost:  0.00021412721
Target cost:  0.00021484107
Target cost:  0.00021555695
Target cost:  0.00021625757
Target cost:  0.00021696107
Target cost:  0.00021766123
Target cost:  0.00021836249
Target cost:  0.00021906273
Target cost:  0.00021976816
Target cost:  0.00022047483
Target cost:  0.00022118505
Target cost:  0.00022189772
Target cost:  0.00022261555
Target cost:  0.0002233239
Target cost:  0.0002240397
Target cost:  0.00022475743
Target cost:  0.00022547714
Target cost:  0.0002262022
Target cost:  0.00022692146
Target cost:  0.00022763845
Target cost:  0.0002283686
Target cost:  0.00022910141
Target cost:  0.0002298363

Target cost:  0.0007455242
Target cost:  0.0007527181
Target cost:  0.00075785053
Target cost:  0.00076289725
Target cost:  0.00076784386
Target cost:  0.0007727158
Target cost:  0.000776608
Target cost:  0.00078030874
Target cost:  0.0007814156
Target cost:  0.00079049944
Target cost:  0.00079484086
Target cost:  0.0007998189
Target cost:  0.0008017605
Target cost:  0.0008098107
Target cost:  0.00081385433
Target cost:  0.0008187831
Target cost:  0.00082223525
Target cost:  0.00082992855
Target cost:  0.00083477574
Target cost:  0.0008402648
Target cost:  0.000844985
Target cost:  0.0008501639
Target cost:  0.0008543918
Target cost:  0.000859438
Target cost:  0.00086120685
Target cost:  0.00087041693
Target cost:  0.00087746436
Target cost:  0.0008831791
Target cost:  0.00088859734
Target cost:  0.00089297607
Target cost:  0.00089693285
Target cost:  0.00090151344
Target cost:  0.0009068031
Target cost:  0.0009109827
Target cost:  0.0009189225
Target cost:  0.0009270603
Target cost:  

In [10]:
'''
Un ataque adversario intenta engañar al modelo, pero siempre tratando que las modificaciones sobre la imagen
sean imperceptibles a la vista. Para eso definimos las variables de perturbación que van a indicar cuanto
dejamos que se modifiquen los valores de la imagen
'''
pert = 0.01
max_pert = x + pert
min_pert = x - pert
cost_with_pert = 0.0
# Entrenamiento con pert: mientras que el coste sea menor que 0.95 sigue entrenando
while cost_with_pert < 0.95:
    '''
    El 0 indica que estamos en entrenamiento, es la entrada a la función k.learning_phase()
    gr son las direcciones de hacia dónde tenemos que modificar los píxeles de la imágen para 
    optimizar el sistema de modificación de imágenes
    '''
    gr, cost_with_pert = optimize_gradients([adv_pert, 0]) 
    # Sumamos a nuestra imagen los valores de los gradientes de forma que la modificamos en las direcciones deseadas
    adv_pert += gr
    '''
    Para que la moficiación en la imagen sea imperceptible. Clip hace que cuando un valor pasa el umbral
    lo deja en el valor máximo o mínimo definido
    '''
    adv_pert = np.clip(adv_pert, min_pert, max_pert)
    # Esta siguiente línea es para que al modificar los píxeles de la imagen no se salgan del rango [-1, 1]
    adv_pert = np.clip(adv_pert, -1, 1)
    print('Target cost PERT: ', cost_with_pert)
hacked_img_pert = np.copy(adv_pert)


Target cost PERT:  5.0135204e-06
Target cost PERT:  5.014187e-06
Target cost PERT:  5.015791e-06
Target cost PERT:  5.01773e-06
Target cost PERT:  5.0198637e-06
Target cost PERT:  5.0220897e-06
Target cost PERT:  5.0242634e-06
Target cost PERT:  5.0265458e-06
Target cost PERT:  5.028793e-06
Target cost PERT:  5.03103e-06
Target cost PERT:  5.0332587e-06
Target cost PERT:  5.035538e-06
Target cost PERT:  5.0378017e-06
Target cost PERT:  5.0400436e-06
Target cost PERT:  5.04233e-06
Target cost PERT:  5.044628e-06
Target cost PERT:  5.046902e-06
Target cost PERT:  5.0491685e-06
Target cost PERT:  5.0514395e-06
Target cost PERT:  5.0536623e-06
Target cost PERT:  5.0559397e-06
Target cost PERT:  5.058142e-06
Target cost PERT:  5.0603744e-06
Target cost PERT:  5.062626e-06
Target cost PERT:  5.0648705e-06
Target cost PERT:  5.06712e-06
Target cost PERT:  5.06937e-06
Target cost PERT:  5.0716217e-06
Target cost PERT:  5.0738654e-06
Target cost PERT:  5.0761428e-06
Target cost PERT:  5.078383e

Target cost PERT:  5.583526e-06
Target cost PERT:  5.5859036e-06
Target cost PERT:  5.588223e-06
Target cost PERT:  5.5905766e-06
Target cost PERT:  5.5928767e-06
Target cost PERT:  5.595222e-06
Target cost PERT:  5.597564e-06
Target cost PERT:  5.5999376e-06
Target cost PERT:  5.60226e-06
Target cost PERT:  5.6045765e-06
Target cost PERT:  5.606958e-06
Target cost PERT:  5.6092654e-06
Target cost PERT:  5.6116232e-06
Target cost PERT:  5.6139606e-06
Target cost PERT:  5.6162835e-06
Target cost PERT:  5.6186736e-06
Target cost PERT:  5.620961e-06
Target cost PERT:  5.623312e-06
Target cost PERT:  5.6256436e-06
Target cost PERT:  5.6279555e-06
Target cost PERT:  5.6302656e-06
Target cost PERT:  5.6326e-06
Target cost PERT:  5.6348977e-06
Target cost PERT:  5.637212e-06
Target cost PERT:  5.6394897e-06
Target cost PERT:  5.6418316e-06
Target cost PERT:  5.6441404e-06
Target cost PERT:  5.646465e-06
Target cost PERT:  5.6487675e-06
Target cost PERT:  5.65111e-06
Target cost PERT:  5.65340

Target cost PERT:  6.2178706e-06
Target cost PERT:  6.220518e-06
Target cost PERT:  6.223225e-06
Target cost PERT:  6.225885e-06
Target cost PERT:  6.228546e-06
Target cost PERT:  6.231213e-06
Target cost PERT:  6.23391e-06
Target cost PERT:  6.2366034e-06
Target cost PERT:  6.2392987e-06
Target cost PERT:  6.2420017e-06
Target cost PERT:  6.244705e-06
Target cost PERT:  6.2473964e-06
Target cost PERT:  6.250124e-06
Target cost PERT:  6.2528034e-06
Target cost PERT:  6.255536e-06
Target cost PERT:  6.2582326e-06
Target cost PERT:  6.2609383e-06
Target cost PERT:  6.263663e-06
Target cost PERT:  6.2663985e-06
Target cost PERT:  6.2691306e-06
Target cost PERT:  6.2718386e-06
Target cost PERT:  6.2745826e-06
Target cost PERT:  6.2773006e-06
Target cost PERT:  6.2800395e-06
Target cost PERT:  6.2827744e-06
Target cost PERT:  6.2854647e-06
Target cost PERT:  6.2882523e-06
Target cost PERT:  6.2909703e-06
Target cost PERT:  6.2937165e-06
Target cost PERT:  6.2964295e-06
Target cost PERT:  6.

Target cost PERT:  6.965053e-06
Target cost PERT:  6.968085e-06
Target cost PERT:  6.9711327e-06
Target cost PERT:  6.97415e-06
Target cost PERT:  6.9772586e-06
Target cost PERT:  6.9802863e-06
Target cost PERT:  6.9833145e-06
Target cost PERT:  6.9863904e-06
Target cost PERT:  6.9893676e-06
Target cost PERT:  6.992392e-06
Target cost PERT:  6.995442e-06
Target cost PERT:  6.9985176e-06
Target cost PERT:  7.0014917e-06
Target cost PERT:  7.004514e-06
Target cost PERT:  7.00755e-06
Target cost PERT:  7.010556e-06
Target cost PERT:  7.0135898e-06
Target cost PERT:  7.0166293e-06
Target cost PERT:  7.0196456e-06
Target cost PERT:  7.022641e-06
Target cost PERT:  7.025667e-06
Target cost PERT:  7.028645e-06
Target cost PERT:  7.031661e-06
Target cost PERT:  7.034678e-06
Target cost PERT:  7.0377237e-06
Target cost PERT:  7.040755e-06
Target cost PERT:  7.043769e-06
Target cost PERT:  7.0467995e-06
Target cost PERT:  7.0497804e-06
Target cost PERT:  7.0528017e-06
Target cost PERT:  7.055846

Target cost PERT:  7.77516e-06
Target cost PERT:  7.778565e-06
Target cost PERT:  7.782012e-06
Target cost PERT:  7.785419e-06
Target cost PERT:  7.788895e-06
Target cost PERT:  7.792314e-06
Target cost PERT:  7.795697e-06
Target cost PERT:  7.799108e-06
Target cost PERT:  7.80255e-06
Target cost PERT:  7.806009e-06
Target cost PERT:  7.809486e-06
Target cost PERT:  7.812913e-06
Target cost PERT:  7.816409e-06
Target cost PERT:  7.819836e-06
Target cost PERT:  7.8233115e-06
Target cost PERT:  7.8268195e-06
Target cost PERT:  7.830345e-06
Target cost PERT:  7.833842e-06
Target cost PERT:  7.837334e-06
Target cost PERT:  7.840876e-06
Target cost PERT:  7.844465e-06
Target cost PERT:  7.848091e-06
Target cost PERT:  7.851683e-06
Target cost PERT:  7.855283e-06
Target cost PERT:  7.858896e-06
Target cost PERT:  7.862515e-06
Target cost PERT:  7.866099e-06
Target cost PERT:  7.869706e-06
Target cost PERT:  7.873344e-06
Target cost PERT:  7.87693e-06
Target cost PERT:  7.8806e-06
Target cost

...

Target cost PERT:  0.0043689315
Target cost PERT:  0.017657649
Target cost PERT:  0.06616988
Target cost PERT:  0.0027960264
Target cost PERT:  0.009061989
Target cost PERT:  0.061411347
Target cost PERT:  0.007443698
Target cost PERT:  0.085687034
Target cost PERT:  0.0056654406
Target cost PERT:  0.10758452
Target cost PERT:  0.007424101
Target cost PERT:  0.033828698
Target cost PERT:  0.012531254
Target cost PERT:  0.059466418
Target cost PERT:  0.008205564
Target cost PERT:  0.06961628
Target cost PERT:  0.0052471915
Target cost PERT:  0.03998481
Target cost PERT:  0.039220907
Target cost PERT:  0.014838698
Target cost PERT:  0.13526425
Target cost PERT:  0.0021043324
Target cost PERT:  0.005887295
Target cost PERT:  0.038823403
Target cost PERT:  0.0470473
Target cost PERT:  0.009158526
Target cost PERT:  0.07133894
Target cost PERT:  0.0072794845
Target cost PERT:  0.07749724
Target cost PERT:  0.006762943
Target cost PERT:  0.09338135
Target cost PERT:  0.004384279
Target cost 

Target cost PERT:  0.004445163
Target cost PERT:  0.026531799
Target cost PERT:  0.042838782
Target cost PERT:  0.028728843
Target cost PERT:  0.022347908
Target cost PERT:  0.13612098
Target cost PERT:  0.0011658372
Target cost PERT:  0.002355384
Target cost PERT:  0.008309857
Target cost PERT:  0.05640285
Target cost PERT:  0.01049503
Target cost PERT:  0.09964473
Target cost PERT:  0.004868334
Target cost PERT:  0.061149802
Target cost PERT:  0.012959662
Target cost PERT:  0.060001396
Target cost PERT:  0.009290347
Target cost PERT:  0.15062782
Target cost PERT:  0.002534199
Target cost PERT:  0.0069990354
Target cost PERT:  0.03496349
Target cost PERT:  0.017077615
Target cost PERT:  0.045957044
Target cost PERT:  0.026745988
Target cost PERT:  0.012927798
Target cost PERT:  0.09657453
Target cost PERT:  0.007750894
Target cost PERT:  0.11598386
Target cost PERT:  0.0033140338
Target cost PERT:  0.016869308
Target cost PERT:  0.09465569
Target cost PERT:  0.0025065471
Target cost P

Target cost PERT:  0.049150817
Target cost PERT:  0.005341305
Target cost PERT:  0.023688573
Target cost PERT:  0.07109113
Target cost PERT:  0.0031764577
Target cost PERT:  0.0128105525
Target cost PERT:  0.076871485
Target cost PERT:  0.0046732957
Target cost PERT:  0.019800825
Target cost PERT:  0.044611577
Target cost PERT:  0.006037476
Target cost PERT:  0.05017945
Target cost PERT:  0.021081338
Target cost PERT:  0.037168857
Target cost PERT:  0.041782867
Target cost PERT:  0.015718948
Target cost PERT:  0.083662026
Target cost PERT:  0.008890048
Target cost PERT:  0.146576
Target cost PERT:  0.0035896231
Target cost PERT:  0.017453698
Target cost PERT:  0.108040504
Target cost PERT:  0.0029698045
Target cost PERT:  0.008910469
Target cost PERT:  0.058687042
Target cost PERT:  0.0072247726
Target cost PERT:  0.0858901
Target cost PERT:  0.0039496534
Target cost PERT:  0.027882384
Target cost PERT:  0.03161311
Target cost PERT:  0.029524323
Target cost PERT:  0.023060326
Target co

Target cost PERT:  0.0034469925
Target cost PERT:  0.016246615
Target cost PERT:  0.057986684
Target cost PERT:  0.004071977
Target cost PERT:  0.019294493
Target cost PERT:  0.07082683
Target cost PERT:  0.0036782399
Target cost PERT:  0.021203967
Target cost PERT:  0.0668289
Target cost PERT:  0.005651639
Target cost PERT:  0.047592845
Target cost PERT:  0.033079218
Target cost PERT:  0.025487432
Target cost PERT:  0.0602407
Target cost PERT:  0.016078271
Target cost PERT:  0.038252395
Target cost PERT:  0.019398239
Target cost PERT:  0.10122737
Target cost PERT:  0.0020277859
Target cost PERT:  0.0046571638
Target cost PERT:  0.02785236
Target cost PERT:  0.058776163
Target cost PERT:  0.0055469684
Target cost PERT:  0.02742395
Target cost PERT:  0.0877333
Target cost PERT:  0.007928204
Target cost PERT:  0.07987267
Target cost PERT:  0.003600382
Target cost PERT:  0.017765388
Target cost PERT:  0.12568787
Target cost PERT:  0.0029805521
Target cost PERT:  0.01086524
Target cost PER

KeyboardInterrupt: 

In [ ]:
'''
Una vez que se ha entrenado el sistema vamos a ver cómo es la imagen que hemos modificado
Volvemos a poner la imagen en el formato inicial: las intensidades [0-255]
'''

plt.imshow(x)
plt.title('Imagen original')
plt.show()

hacked_img /= 2
hacked_img += 0.5
hacked_img *= 255
plt.imshow(hacked_img[0].astype(np.uint8))
plt.title('Imagen modificada sin control de perturbaciones')
plt.show()

hacked_img_pert /= 2
hacked_img_pert += 0.5
hacked_img_pert *= 255
plt.imshow(hacked_img_pert[0].astype(np.uint8))
plt.title('Imagen modificada con control de perturbaciones')
plt.show()


f, axarr = plt.subplots(1,3)
axarr[0,0].imshow(x)
axarr[0,1].imshow(hacked_img[0].astype(np.uint8))
axarr[0,2].imshow(hacked_img_pert[0].astype(np.uint8))